# Generación de datos en Parquet

In [6]:
#!pip install fsspec

In [7]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import dask
from dask import dataframe as dd
import platform
from glob import glob
import duckdb

In [8]:
paths = glob("drive/MyDrive/Ciclos/2023-1/Big Data/Labs/data/tbl/*")

headers = {
    "customer": ["CUSTKEY", "NAME", "ADDRESS", "NATIONKEY", "PHONE", "ACCTBAL", "MKTSEGMENT", "COMMENT"],
    "region": ["REGIONKEY", "NAME", "COMMENT"],
    "nation": ["NATIONKEY", "NAME", "REGIONKEY", "COMMENT"],
    "lineitem": ["ORDERKEY", "PARTKEY", "SUPPKEY", "LINENUMBER", "QUANTITY", "EXTENDEDPRICE", "DISCOUNT", "TAX", "RETURNFLAG", "LINESTATUS", "SHIPDATE", "COMMITDATE", "RECEIPTDATE", "SHIPINSTRUCT", "SHIPMODE", "COMMENT"],
    "orders": ["ORDERKEY", "CUSTKEY", "ORDERSTATUS", "TOTALPRICE", "ORDERDATE", "ORDERPRIORITY", "CLERK", "SHIPPRIORITY", "COMMENT"],
    "part": ["PARTKEY", "NAME", "MFGR", "BRAND", "TYPE", "SIZE", "CONTAINER", "RETAILPRICE", "COMMENT"],
    "partsupp": ["PARTKEY", "SUPPKEY", "AVAILQTY", "SUPPLYCOST", "COMMENT"],
    "supplier": ["SUPPKEY", "NAME", "ADDRESS", "NATIONKEY", "PHONE", "ACCTBAL", "COMMENT"]
}

partitions = {
    "customer": ["NATIONKEY"],
    "region": [],
    "nation": [],
    "lineitem": ["SHIPMODE"],
    "orders": ["ORDERSTATUS"],
    "part": ["BRAND"],
    "partsupp": [],
    "supplier": ["NATIONKEY"]
}

prefixes = ['c', 'r', 'n', 'l', 'o', 'p', 'ps', 's']

for i, key in enumerate(headers):
  headers[key] = list(map(lambda x: prefixes[i]+'_'+x.lower(), headers[key]))
  partitions[key] = list(map(lambda x: prefixes[i]+'_'+x.lower(), partitions[key]))


In [9]:
tables = []

for path in paths:
  ps = path.split('/')
  name = ps[len(ps)-1][:-4]
  table = pd.read_table(path, low_memory=False, delimiter='|', names=headers[name]+["del"])
  table = table.drop(["del"], axis=1)
  tables.append(table)

In [10]:
tables[0].info(verbose=True)
tables[0].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ps_partkey     800000 non-null  int64  
 1   ps_suppkey     800000 non-null  int64  
 2   ps_availqty    800000 non-null  int64  
 3   ps_supplycost  800000 non-null  float64
 4   ps_comment     800000 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 30.5+ MB


,ps_partkey,ps_suppkey,ps_availqty,ps_supplycost,ps_comment
0,1,2,3325,771.64,", even theodolites. regular, final theodolites..."
1,1,2502,8076,993.49,ven ideas. quickly even packages print. pendin...
2,1,5002,3956,337.09,after the fluffily ironic deposits? blithely s...
3,1,7502,4069,357.84,"al, regular dependencies serve carefully after..."
4,2,3,8895,378.49,nic accounts. final accounts sleep furiously a...


In [11]:
for i, table in enumerate(tables):
  ps = paths[i].split('/')
  name = ps[len(ps)-1][:-4]
  table.to_parquet(
    path='drive/MyDrive/Ciclos/2023-1/Big Data/Labs/data/parquet/'+f'{name}.parquet',
    engine='pyarrow',
    compression='gzip',
  )

In [12]:
for i, table in enumerate(tables):
  ps = paths[i].split('/')
  name = ps[len(ps)-1][:-4]
  table2 = dd.from_pandas(table, npartitions=1)
  table2.to_parquet(
    path='drive/MyDrive/Ciclos/2023-1/Big Data/Labs/data/parquet/'+f'{name}/',
    engine='pyarrow',
    compression='snappy',
    partition_on=partitions[name]
  )

# Lectura de datos

## Data sin particionar

In [13]:
pq_path = "drive/MyDrive/Ciclos/2023-1/Big Data/Labs/data/parquet/"

customer = pd.read_parquet(pq_path + "customer" + ".parquet")
region = pd.read_parquet(pq_path + "region" + ".parquet")
nation = pd.read_parquet(pq_path + "nation" + ".parquet")
lineitem = pd.read_parquet(pq_path + "lineitem" + ".parquet")
orders = pd.read_parquet(pq_path + "orders" + ".parquet")
part = pd.read_parquet(pq_path + "part" + ".parquet")
partsupp = pd.read_parquet(pq_path + "partsupp" + ".parquet")
supplier = pd.read_parquet(pq_path + "supplier" + ".parquet")

In [14]:
query = """select
	l_orderkey,
	sum(l_extendedprice * (1 - l_discount)) as revenue,
	o_orderdate,
	o_shippriority
from
	customer,
	orders,
	lineitem
where
	c_mktsegment = 'BUILDING'
	and c_custkey = o_custkey
	and l_orderkey = o_orderkey
	and o_orderdate < date '1995-03-15'
	and l_shipdate > date '1995-03-15'
group by
	l_orderkey,
	o_orderdate,
	o_shippriority
order by
	revenue desc,
	o_orderdate
limit 10;"""
duckdb.sql(query)

┌────────────┬────────────────────┬─────────────┬────────────────┐
│ l_orderkey │      revenue       │ o_orderdate │ o_shippriority │
│   int64    │       double       │   varchar   │     int64      │
├────────────┼────────────────────┼─────────────┼────────────────┤
│    2456423 │        406181.0111 │ 1995-03-05  │              0 │
│    3459808 │ 405838.69889999996 │ 1995-03-04  │              0 │
│     492164 │         390324.061 │ 1995-02-19  │              0 │
│    1188320 │        384537.9359 │ 1995-03-09  │              0 │
│    2435712 │ 378673.05580000003 │ 1995-02-26  │              0 │
│    4878020 │        378376.7952 │ 1995-03-12  │              0 │
│    5521732 │        375153.9215 │ 1995-03-13  │              0 │
│    2628192 │ 373133.30939999997 │ 1995-02-22  │              0 │
│     993600 │        371407.4595 │ 1995-03-05  │              0 │
│    2300070 │  367371.1452000001 │ 1995-03-13  │              0 │
├────────────┴────────────────────┴─────────────┴─────────────

## Data particionada

In [15]:
customer = dd.read_parquet(pq_path + "customer" + "/", engine='pyarrow')
region = dd.read_parquet(pq_path + "region" + "/", engine='pyarrow')
nation = dd.read_parquet(pq_path + "nation" + "/", engine='pyarrow')
lineitem = dd.read_parquet(pq_path + "lineitem" + "/", engine='pyarrow')
orders = dd.read_parquet(pq_path + "orders" + "/", engine='pyarrow')
part = dd.read_parquet(pq_path + "part" + "/", engine='pyarrow')
partsupp = dd.read_parquet(pq_path + "partsupp" + "/", engine='pyarrow')
supplier = dd.read_parquet(pq_path + "supplier" + "/", engine='pyarrow')

In [16]:
part.compute()

,p_partkey,p_name,p_mfgr,p_type,p_size,p_container,p_retailprice,p_comment,p_brand
6,7,moccasin green thistle khaki floral,Manufacturer#1,SMALL PLATED COPPER,45,SM BAG,907.00,lyly. ex,Brand#11
17,18,turquoise indian lemon lavender misty,Manufacturer#1,SMALL BURNISHED STEEL,42,JUMBO PACK,918.01,s cajole slyly a,Brand#11
45,46,honeydew turquoise aquamarine spring tan,Manufacturer#1,STANDARD POLISHED TIN,45,WRAP CASE,946.04,the blithely unusual,Brand#11
59,60,snow spring sandy olive tomato,Manufacturer#1,LARGE POLISHED COPPER,27,JUMBO CASE,960.06,integ,Brand#11
67,68,bisque ivory mint purple almond,Manufacturer#1,PROMO ANODIZED STEEL,10,WRAP BOX,968.06,eposits shall h,Brand#11
...,...,...,...,...,...,...,...,...,...
199915,199916,sky floral beige cornflower black,Manufacturer#5,SMALL BURNISHED COPPER,21,SM BOX,2015.91,oost quickly caref,Brand#55
199932,199933,gainsboro thistle seashell sandy midnight,Manufacturer#5,LARGE BRUSHED TIN,37,LG PACK,2032.93,are fur,Brand#55
199974,199975,snow misty medium ivory peru,Manufacturer#5,MEDIUM PLATED COPPER,42,SM BOX,2074.97,wake carefull,Brand#55
199988,199989,indian medium mint snow gainsboro,Manufacturer#5,ECONOMY PLATED TIN,11,JUMBO DRUM,2088.98,ven requests. ironi,Brand#55
